Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import math

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.790301
Minibatch accuracy: 18.8%
Validation accuracy: 10.9%
Minibatch loss at step 50: 1.763833
Minibatch accuracy: 25.0%
Validation accuracy: 51.5%
Minibatch loss at step 100: 0.955613
Minibatch accuracy: 68.8%
Validation accuracy: 69.2%
Minibatch loss at step 150: 0.494459
Minibatch accuracy: 87.5%
Validation accuracy: 77.2%
Minibatch loss at step 200: 1.256703
Minibatch accuracy: 68.8%
Validation accuracy: 77.5%
Minibatch loss at step 250: 1.368745
Minibatch accuracy: 56.2%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.282849
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 350: 1.019160
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Minibatch loss at step 400: 0.222610
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.532066
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.808980
Minibatch accuracy: 75.0%
Validation accuracy: 81.7%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pooling = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pooling, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pooling = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = pooling.get_shape().as_list()
    print ('shape :',shape)
    reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

shape : [16, 7, 7, 16]
shape : [10000, 7, 7, 16]
shape : [10000, 7, 7, 16]


In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.791054
Minibatch accuracy: 18.8%
Validation accuracy: 9.7%
Minibatch loss at step 50: 2.049822
Minibatch accuracy: 25.0%
Validation accuracy: 33.3%
Minibatch loss at step 100: 1.604667
Minibatch accuracy: 31.2%
Validation accuracy: 57.2%
Minibatch loss at step 150: 0.659051
Minibatch accuracy: 81.2%
Validation accuracy: 73.5%
Minibatch loss at step 200: 1.601996
Minibatch accuracy: 62.5%
Validation accuracy: 75.2%
Minibatch loss at step 250: 1.483264
Minibatch accuracy: 56.2%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.255188
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.828919
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
Minibatch loss at step 400: 0.293031
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 450: 0.565657
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 500: 1.142270
Minibatch accuracy: 68.8%
Validation accuracy: 81.6%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
batch_size = 128
patch_size = 3
depth1 = 64
depth2 = 64
num_hidden1 = 1024
num_hidden2 = 512
num_hidden3 = 256

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=math.sqrt(2./(patch_size*patch_size*num_channels))))
  layer1_biases = tf.Variable(tf.zeros([depth1]))

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=math.sqrt(2./(patch_size*patch_size*depth1))))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))


  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth2, num_hidden1], stddev=math.sqrt(2./(image_size // 4 * image_size // 4 * depth2))))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))

  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=math.sqrt(2./num_hidden1)))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))

  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_hidden3], stddev=math.sqrt(2./num_hidden2)))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden3]))

  layer6_weights = tf.Variable(tf.truncated_normal(
      [num_hidden3, num_labels], stddev=math.sqrt(2./num_hidden3)))
  layer6_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, is_train=True):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pooling = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    conv = tf.nn.conv2d(pooling, layer2_weights, [1, 1, 1, 1], padding='SAME')    
    hidden = tf.nn.relu(conv + layer2_biases)
    pooling = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    shape = pooling.get_shape().as_list()
    reshape = tf.reshape(pooling, [shape[0], shape[1] * shape[2] * shape[3]])
 
    if is_train : reshape = tf.nn.dropout(reshape, 0.9)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    if is_train : hidden = tf.nn.dropout(hidden, 0.7)
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    if is_train : hidden = tf.nn.dropout(hidden, 0.7)
    hidden = tf.nn.relu(tf.matmul(hidden, layer5_weights) + layer5_biases)
    if is_train : hidden = tf.nn.dropout(hidden, 0.7)
    return tf.matmul(hidden, layer6_weights) + layer6_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  regularizers = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                  tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                  tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
                  tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases) +
                  tf.nn.l2_loss(layer5_weights) + tf.nn.l2_loss(layer5_biases) +
                  tf.nn.l2_loss(layer6_weights) + tf.nn.l2_loss(layer6_biases))
  loss += 1e-4 * regularizers

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, is_train=False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, is_train=False))

In [10]:
num_steps = 40001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 4000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 8.346124
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Minibatch loss at step 4000: 0.668426
Minibatch accuracy: 85.2%
Validation accuracy: 87.0%
Minibatch loss at step 8000: 0.653367
Minibatch accuracy: 86.7%
Validation accuracy: 88.9%
Minibatch loss at step 12000: 0.484248
Minibatch accuracy: 90.6%
Validation accuracy: 89.8%
Minibatch loss at step 16000: 0.623545
Minibatch accuracy: 85.9%
Validation accuracy: 90.3%
Minibatch loss at step 20000: 0.585054
Minibatch accuracy: 89.8%
Validation accuracy: 90.8%
Minibatch loss at step 24000: 0.476550
Minibatch accuracy: 93.0%
Validation accuracy: 91.1%
Minibatch loss at step 28000: 0.575180
Minibatch accuracy: 90.6%
Validation accuracy: 91.2%
Minibatch loss at step 32000: 0.482539
Minibatch accuracy: 91.4%
Validation accuracy: 91.2%
Minibatch loss at step 36000: 0.575349
Minibatch accuracy: 93.0%
Validation accuracy: 91.6%
Minibatch loss at step 40000: 0.444323
Minibatch accuracy: 92.2%
Validation